# Forced Aperture Photometry

***

## Kernel Information

To run this notebook, please select the "Roman Calibration" kernel at the top right of your window.

## Imports
We are using standard libraries from the Numpy and Astropy stack.

- `asdf` opens the Roman WFI data in the Advanced Scientific Data Format (ASDF). 
- `photutils` is an Astropy-affiliated package for photometry.
- `roman_datamodels` handles and validates data using schemas that define the expected structure of Roman data.
- `s3fs` streams data from Simple Storage Service (S3) buckets on Amazon Web Services (AWS).

In [ ]:
import asdf
from astropy.io import fits
from astropy.table import Table
import copy
import matplotlib.pyplot as plt
import numpy as np
import photutils
import roman_datamodels as rdm
import s3fs

## Introduction to aperture photometry
This notebook explains how to perform **forced aperture photometry** (also known as *forced photometry* or *aperture photometry*). Aperture photometry is useful for measuring the integrated fluxes for a set of specified source positions and aperture sizes. This technique is often valuable for extracting fluxes of known sources when it is impractical to fit for their positions and light distributions, or when simplicity and speed are high priorities. For example, aperture photometry can be used for:

- **Faint sources.** If a source is too faint, then it can be difficult to fit for the source centroid, light profile, and flux.
- **Sources detected at other wavelengths.** If a source is detected in a given bandpass, and the source size is expected to be the same across wavelengths, then it can be useful to define one aperture and measure the source fluxes across multiple filters.
- **Time-series evolution of sources.** If a source brightness is decreasing over time, then we can use earlier observations obtained when the source was bright to specify an aperture and extract the flux in a time series.

Here, we cover a simple example using the `photutils` package to extract fluxes from a catalog of known sources (stars and galaxies).

***

## Loading data

We obtain data from S3 buckets by using `s3fs`. For more information about accessing and streaming data on the cloud, please refer to the Data Discovery and Access notebook tutorial.

### WFI image

We load a simulated Roman WFI image in the F158 filter generated with [Roman I-Sim](https://romanisim.readthedocs.io/) from a mock source catalog of galaxies and stars described in detail below. This (4088, 4088) pixel image covers a single detector: Sensor Chip Assembly (SCA) 01. It is a Level 2 image, meaning that the data were processed to flag and/or correct for detector-level effects (e.g., saturation, classic non-linearity, etc.), and that resultants were fitted into a count rate image in units of Data Numbers (DN) per second.

In [ ]:
asdf_dir_uri = 's3://roman-sci-test-data-prod-summer-beta-test/'
fs = s3fs.S3FileSystem()

asdf_file_uri = asdf_dir_uri + 'ROMANISIM/DENSE_REGION/R0.5_DP0.5_PA0/r0000101001001001001_01101_0001_WFI01_cal.asdf'

with fs.open(asdf_file_uri, 'rb') as f:
    dm = rdm.open(f)
    image = dm.data.value.copy()
    wcs = copy.deepcopy(dm.meta.wcs)

### Source catalog

We also have access to a source catalog that was used to simulate the WFI image. It contains stars and galaxies, which are labeled as `PSF` and `SER` under the column `type`. Source fluxes are available in all WFI filters (`F062`, `F087`, `F106`, `F129`, `F146`, `F158`, `F184`, `F213`) and are sampled from a lognormal distribution. Note that fluxes are all given in *maggies*, which are defined as ${\rm maggie} \equiv 10^{-0.4 m_{AB}}$, for an AB apparent magnitude $m_{AB}$. 

For galaxies, morphological parameters like `n` (Sersic index), `half_light_radius`, `pa` (position angle), and `ba` (axis ratio) are also provided in the catalog. These are sampled according to fiducial (and likely unrealistic) distributions.

In [ ]:
catalog_file_uri = asdf_dir_uri + 'ROMANISIM/CATALOGS_SCRIPTS/fullcat_101M_pared_ra0.50_dec0.50_WFI01.ecsv'
with fs.open(catalog_file_uri, 'rb') as catalog_file_stream:
    cat = Table.read(catalog_file_stream, format='ascii.ecsv')

We can also display the first five rows and all columns of the catalog:

In [ ]:
cat[:5]

We can convert (RA, Dec) to (x, y) positions on the WFI01 detector.

In [ ]:
x_cat, y_cat = wcs.world_to_array_index_values(cat["ra"], cat["dec"])

It might be helpful to quantify the number of sources there are before we try to visualize them! We first create a mask of sources that actually fall on the detector, and then divide the remaining sources into stars and galaxies.

In [ ]:
on_detector_mask = (x_cat >= 0) & (x_cat <= 4095) & (y_cat >= 0) & (y_cat <= 4095)
print(f"Number of sources on detector: {sum(on_detector_mask)}")

stars = cat[on_detector_mask & (cat["type"] == "PSF")]
gals = cat[on_detector_mask & (cat["type"] == "SER")]
print(f"Number of stars: {len(stars)}\nNumber of galaxies: {len(gals)}")

Note that there are two orders of magnitude more stars than galaxies. We can now plot the distribution of source fluxes using a histogram. 

In [ ]:
plt.figure(figsize=(4, 3), dpi=150)
plt.hist(np.log10(stars["F158"].value), bins=50, range=[-10, -5], log=True, label="Stars")
plt.hist(np.log10(gals["F158"].value), bins=50, range=[-10, -5], log=True, label="Galaxies")

plt.xlabel("$\log_{10}$(F158 flux [maggies])")
plt.ylabel("Number of sources")
plt.legend()

plt.grid(alpha=0.15)

## Forced aperture photometry

For simplicity, we will perform aperture photometry on a sub-sample of about 1000 stars and 1000 galaxies.

### Selection of bright stars and galaxies

We will define a limiting magnitude of $3 \times 10^{-9}$ maggies in the F158 band, and then select every 100th star and every galaxy. This will give us approximately the same number of stars as galaxies.

From the catalogs, we now know the positions of every selected star and galaxy. We can define set aperture radii in units of pixels; we choose radii of 3 pixels for stars and 5 pixels for galaxies.

In [ ]:
limiting_flux = 3e-9

In [ ]:
bright_stars = stars[(stars["F158"] > limiting_flux)][::100]
print(f"We have selected {len(bright_stars)} bright stars.")

In [ ]:
star_positions = [(y, x) for x, y in zip(*wcs.world_to_array_index_values(bright_stars["ra"], bright_stars["dec"]))]
star_apertures = photutils.aperture.CircularAperture(
    positions=star_positions, 
    r=3
)

In [ ]:
bright_galaxies = gals[gals["F158"] > limiting_flux]
print(f"We have selected {len(bright_galaxies)} bright galaxies.")

In [ ]:
galaxy_positions = [(y, x) for x, y in zip(*wcs.world_to_array_index_values(bright_galaxies["ra"], bright_galaxies["dec"]))]
galaxy_apertures = photutils.aperture.CircularAperture(
    positions=galaxy_positions, 
    r=5
)

### Visualize 1/16th of simulated image

In [ ]:
plt.figure(figsize=(9, 9), dpi=150)

# show the simulated image
plt.imshow(image, origin='lower', vmin=0, vmax=12, cmap="gray_r", )

# plot circles over bright galaxies and stars
star_apertures.plot(color="C0")
galaxy_apertures.plot(color="C1")

# zoom in on 1/16th of the image
plt.xlim(1024, 1536)
plt.ylim(1024, 1536)

plt.axis("off")
plt.show()

### Aperture photometry with `photutils`

We now perform aperture photometry on the selected sources:

In [ ]:
star_phot = photutils.aperture.aperture_photometry(image, star_apertures)
star_phot[:5]

In [ ]:
galaxy_phot = photutils.aperture.aperture_photometry(image, galaxy_apertures)
galaxy_phot[:5]

Let's evaluate our results by plotting the measured fluxes versus the simulated catalog fluxes. The blue points showing stars are follow a much tighter relation, which is expected given their compact sizes and smaller apertures. Meanwhile, galaxies require larger apertures, but still lose flux in some cases (for brighter galaxies) and/or are contaminated by other surrounding sources (often more noticeable for fainter galaxies).

In [ ]:
plt.figure(figsize=(5, 5), dpi=150)
plt.scatter(bright_stars["F158"], star_phot["aperture_sum"], s=5, c="C0", label="Stars")
plt.scatter(bright_galaxies["F158"], galaxy_phot["aperture_sum"], s=5, c="C1", label="Galaxies")

plt.xscale("log")
plt.yscale("log")
plt.xlim(3e-9, 3e-7)
plt.ylim(6e1, 6e3)

plt.xlabel("F158 true flux (maggies)")
plt.ylabel("F158 Aperture flux (DN/s)")
plt.legend(loc="lower right", fontsize=12)

plt.grid(alpha=0.15)

## Aditional Resources
The [`photutils` documentation](https://photutils.readthedocs.io/en/stable/) has additional tutorials for detecting and fitting sources in images:

- [Aperture photometry](https://photutils.readthedocs.io/en/stable/aperture.html)
- [Background estimation](https://photutils.readthedocs.io/en/stable/background.html)
- [Source detection](https://photutils.readthedocs.io/en/stable/detection.html)
- [PSF photometry](https://photutils.readthedocs.io/en/stable/psf.html)

## About this notebook

**Author:** John F. Wu (jowu@stsci.edu) 
**Updated On:** 2024-05-03

***

[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/> 